## Fine Tuning

MNISTデータ（手書き数字）のデータを学習したネットワークの結合荷重をFashonMNISTデータ（服装データ）の初期値として利用

- This program was originally written by Mr. Kanda


In [1]:
import numpy as np
import torch

seed = 100
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [2]:
# Dataset : MNIST

from torchvision import datasets, transforms
import torch.utils as utils

transform = transforms.Compose([
    transforms.ToTensor()])

dataset_train_mnist = datasets.MNIST(
    '~/mnist', 
    train=True, 
    download=True, 
    transform=transform)
dataset_test_mnist  = datasets.MNIST(
    '~/mnist', 
    train=False, 
    download=True, 
    transform=transform)

batch_size = 1000

dataloader_train_mnist = utils.data.DataLoader(dataset_train_mnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)
dataloader_test_mnist  = utils.data.DataLoader(dataset_test_mnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)

Extracting /home/kurita/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /home/kurita/mnist/MNIST/raw


Extracting /home/kurita/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /home/kurita/mnist/MNIST/raw



Extracting /home/kurita/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/kurita/mnist/MNIST/raw


Extracting /home/kurita/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/kurita/mnist/MNIST/raw
Processing...
Done!


/home/kurita/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629395347/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# Network

import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv = nn.Sequential(OrderedDict([
        ("conv1", nn.Conv2d(1, 4, kernel_size=5)),
        ("relu1", nn.ReLU()),
        ("pool1", nn.MaxPool2d(2)),
        ("conv2", nn.Conv2d(4, 8, kernel_size=5)),
        ("relu2", nn.ReLU()),
        ("pool2", nn.MaxPool2d(2)),
    ]))
    self.fc = nn.Sequential(OrderedDict([
        ("fc1"  , nn.Linear(8 * 4 * 4, 100)),
        ("relu1", nn.ReLU()),
        ("fc2"  , nn.Linear(100, 10)),
    ]))

  def forward(self, x1):
    x2 = self.conv(x1)
    x3 = x2.view(x2.size()[0], -1)
    y  = self.fc(x3)
    return y

In [4]:
# Training

from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print(model)

for i in range(20):
  print('EPOCH: {epo:}'.format(epo=i+1))

  ### Train ###
  model.train()
  for x, t in dataloader_train_mnist:
    x = x.to(device)
    t = t.to(device)
    model.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

  ### Evaluate the model by using Training Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_train_mnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print('  train loss: {loss:.4f},  train acc: {acc:.4f}'.format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))
    
  ### Evalueate the model by using Test Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_test_mnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print("  test  loss: {loss:.4f},  test  acc: {acc:.4f}".format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))


Net(
  (conv): Sequential(
    (conv1): Conv2d(1, 4, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(4, 8, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (fc1): Linear(in_features=128, out_features=100, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=100, out_features=10, bias=True)
  )
)
EPOCH: 1
  train loss: 0.5910,  train acc: 0.8283
  test  loss: 0.5575,  test  acc: 0.8412
EPOCH: 2
  train loss: 0.3224,  train acc: 0.9044
  test  loss: 0.3014,  test  acc: 0.9095
EPOCH: 3
  train loss: 0.2539,  train acc: 0.9242
  test  loss: 0.2361,  test  acc: 0.9288
EPOCH: 4
  train loss: 0.2057,  train acc: 0.9390
  test  loss: 0.1888,  test  acc: 0.9460
EPOCH: 5
  train loss: 0.1767,  train acc: 0.9456
  test  loss: 0.1616,  test  acc: 0.9512
EPO

In [5]:
# Check Parameter before Fine-Tuning

print(model.conv.conv1.weight[0])
print(model.fc.fc2.weight[0])

tensor([[[-0.0977,  0.0892, -0.1899, -0.1857, -0.1008],
         [ 0.1737, -0.0633, -0.1438, -0.0632, -0.0965],
         [-0.0323,  0.2194,  0.2388,  0.2752,  0.0975],
         [ 0.3187,  0.3776,  0.2905,  0.0600,  0.0574],
         [-0.0225,  0.2458,  0.1401,  0.3167,  0.1199]]], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([ 0.0040,  0.1260, -0.0220, -0.0468,  0.0221, -0.0627, -0.0960, -0.0613,
         0.1272, -0.1395, -0.0673,  0.1380, -0.1924,  0.1084,  0.0065, -0.0804,
         0.0113, -0.1519,  0.1435, -0.0684,  0.0955,  0.0835, -0.0648,  0.0488,
         0.0120,  0.0997, -0.0665, -0.0005, -0.0740, -0.0075,  0.0773,  0.0325,
         0.0312, -0.0902,  0.1036, -0.1669,  0.0570, -0.0090,  0.0493, -0.1672,
         0.0284,  0.0149, -0.0804,  0.0767, -0.1376,  0.1068,  0.0319, -0.1873,
        -0.1084, -0.0346, -0.0412, -0.0341, -0.0450, -0.1729,  0.0479,  0.0408,
        -0.0107,  0.1074, -0.0660, -0.1697, -0.1611, -0.2946,  0.0864, -0.1682,
         0.0937, -0.1178, -0

In [6]:
# Freeze Conv Parameter

for param in model.conv.parameters():
  param.requires_grad = False

for name, param in model.named_parameters():
  print(name, param.requires_grad)

conv.conv1.weight False
conv.conv1.bias False
conv.conv2.weight False
conv.conv2.bias False
fc.fc1.weight True
fc.fc1.bias True
fc.fc2.weight True
fc.fc2.bias True


In [7]:
# Dataset : Fashion-MNIST

from torchvision import datasets, transforms
import torch.utils as utils

transform = transforms.Compose([
    transforms.ToTensor()])

dataset_train_fmnist = datasets.FashionMNIST(
    '~/fashion-mnist', 
    train=True, 
    download=True, 
    transform=transform)
dataset_test_fmnist  = datasets.FashionMNIST(
    '~/fashion-mnist', 
    train=False, 
    download=True, 
    transform=transform)

batch_size = 1000

dataloader_train_fmnist = utils.data.DataLoader(dataset_train_fmnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)
dataloader_test_fmnist  = utils.data.DataLoader(dataset_test_fmnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=4)

Extracting /home/kurita/fashion-mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/kurita/fashion-mnist/FashionMNIST/raw


Extracting /home/kurita/fashion-mnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/kurita/fashion-mnist/FashionMNIST/raw



Extracting /home/kurita/fashion-mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/kurita/fashion-mnist/FashionMNIST/raw


Extracting /home/kurita/fashion-mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/kurita/fashion-mnist/FashionMNIST/raw
Processing...
Done!


In [8]:
# Fine-Tuning

for i in range(20):
  print(f"EPOCH: {i+1}")

  ### Train ###
  model.train()
  for x, t in dataloader_train_fmnist:
    x = x.to(device)
    t = t.to(device)
    model.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    loss.backward()
    optimizer.step()

  ### Evaluate the model by using Training Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_train_fmnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print('  train loss: {loss:.4f},  train acc: {acc:.4f}'.format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))    

  ### Evalueae the model by using Test Samples ###
  model.eval()
  sum_loss = 0.0
  sum_correct = 0
  sum_iter = 0
  for x, t in dataloader_test_fmnist:
    x = x.to(device)
    t = t.to(device)
    y = model(x)
    loss = criterion(y, t)
    _, predicted = y.max(1)
    sum_loss += loss.cpu().detach().numpy()
    sum_correct += (predicted == t).sum().item()
    sum_iter += 1
  print("  test  loss: {loss:.4f},  test  acc: {acc:.4f}".format(loss=sum_loss/sum_iter, acc=sum_correct/(sum_iter*batch_size)))


EPOCH: 1
  train loss: 0.6951,  train acc: 0.7414
  test  loss: 0.7167,  test  acc: 0.7283
EPOCH: 2
  train loss: 0.6280,  train acc: 0.7662
  test  loss: 0.6511,  test  acc: 0.7563
EPOCH: 3
  train loss: 0.5910,  train acc: 0.7804
  test  loss: 0.6159,  test  acc: 0.7712
EPOCH: 4
  train loss: 0.5686,  train acc: 0.7901
  test  loss: 0.5925,  test  acc: 0.7818
EPOCH: 5
  train loss: 0.5571,  train acc: 0.7955
  test  loss: 0.5835,  test  acc: 0.7865
EPOCH: 6
  train loss: 0.5407,  train acc: 0.8030
  test  loss: 0.5665,  test  acc: 0.7929
EPOCH: 7
  train loss: 0.5275,  train acc: 0.8091
  test  loss: 0.5532,  test  acc: 0.8002
EPOCH: 8
  train loss: 0.5272,  train acc: 0.8026
  test  loss: 0.5526,  test  acc: 0.7929
EPOCH: 9
  train loss: 0.5205,  train acc: 0.8111
  test  loss: 0.5485,  test  acc: 0.8024
EPOCH: 10
  train loss: 0.5060,  train acc: 0.8146
  test  loss: 0.5317,  test  acc: 0.8085
EPOCH: 11
  train loss: 0.4987,  train acc: 0.8199
  test  loss: 0.5246,  test  acc: 0.81

In [9]:
# Check Parameter after Fine-Tuning

print(model.conv.conv1.weight[0])
print(model.fc.fc2.weight[0])

tensor([[[-0.0987,  0.0879, -0.1839, -0.1773, -0.0885],
         [ 0.1744, -0.0606, -0.1380, -0.0560, -0.0881],
         [-0.0309,  0.2215,  0.2408,  0.2791,  0.1009],
         [ 0.3180,  0.3777,  0.2913,  0.0617,  0.0589],
         [-0.0238,  0.2449,  0.1394,  0.3165,  0.1186]]], device='cuda:0')
tensor([ 0.0733,  0.0787, -0.1068, -0.0136,  0.0326, -0.0627,  0.2522, -0.0872,
         0.0572, -0.0669, -0.4504,  0.1909, -0.3029,  0.2289, -0.0150, -0.0468,
         0.0593, -0.1911,  0.0770, -0.0960,  0.0730,  0.0593, -0.0585,  0.0385,
         0.0283,  0.1394, -0.0180, -0.0244, -0.2142,  0.0117,  0.2173, -0.0178,
         0.0679, -0.0951,  0.1260, -0.2009,  0.0811, -0.0177,  0.0493, -0.0703,
         0.0029,  0.0346, -0.0475,  0.0728, -0.1105,  0.2052,  0.0417, -0.1982,
        -0.0499,  0.0131, -0.0183, -0.0523, -0.0470, -0.0475,  0.0154,  0.0383,
        -0.0412,  0.1126, -0.0658, -0.1894, -0.1234, -0.2858,  0.0934, -0.1142,
         0.1552, -0.0764,  0.0054,  0.0426, -0.1289,  0.1251,

参考<br>
https://pytorch.org/docs/stable/notes/autograd.html